In [1]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout
from tcn import TCN
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (25,10)
mpl.rcParams['axes.grid'] = False

In [4]:
df = pd.read_csv('House1 with status.csv')
df['Time']= pd.to_datetime(df['Time'])

In [5]:
df.drop('index', axis=1, inplace=True)
raw = df
df.drop('mains_1', axis=1, inplace=True)
df.drop('mains_2', axis=1, inplace=True)

In [6]:
Total = df["refrigerator_3"] + df["dishwaser_4"] + df["microwave_6"] + df["washer_dryer"]
# https://www.kite.com/python/answers/how-to-sum-two-columns-in-a-pandas-dataframe-in-python
df["Total"] = Total
df = df.rename({'refrigerator_3': 'fridge', 'dishwaser_4': 'dw', 'microwave_6': 'mw', 'washer_dryer':'wm', 'fridge_stat':'fridge_status', 'dishwasher_stat': 'dw_status', 'microwave_stat':'mw_status', 'washerdryer_stat':'wm_status'}, axis=1)  # new method
df = df[['Time','Total','fridge','dw','mw','wm','fridge_status','dw_status','mw_status', 'wm_status']]
print('dw --> dishwasher ; mw --> washing machine ; mw --> microwave ')
df.head(4)

dw --> dishwasher ; mw --> washing machine ; mw --> microwave 


,Time,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
0,2011-04-18 13:22:13,11.0,6.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,2011-04-18 13:22:16,11.0,6.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
2,2011-04-18 13:22:20,11.0,6.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,2011-04-18 13:22:23,12.0,6.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.tail(4)

,Time,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
406744,2011-05-24 19:56:23,191.0,187.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
406745,2011-05-24 19:56:27,194.0,190.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
406746,2011-05-24 19:56:30,193.0,189.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0
406747,2011-05-24 19:56:34,190.0,186.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406748 entries, 0 to 406747
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Time           406748 non-null  datetime64[ns]
 1   Total          406748 non-null  float64       
 2   fridge         406748 non-null  float64       
 3   dw             406748 non-null  float64       
 4   mw             406748 non-null  float64       
 5   wm             406748 non-null  float64       
 6   fridge_status  406748 non-null  float64       
 7   dw_status      406748 non-null  float64       
 8   mw_status      406748 non-null  float64       
 9   wm_status      406748 non-null  float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 31.0 MB


In [9]:
raw2 = df

In [10]:
df = df.set_index('Time') 
resample = df.resample('30s')
resample = resample.mean()
resample
df = resample
df.head(5)

,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
Time,,,,,,,,,
2011-04-18 13:22:00,11.200000,6.0,0.200000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:22:30,11.111111,6.0,0.111111,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:00,11.333333,6.0,0.333333,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:30,11.250000,6.0,0.250000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:24:00,11.142857,6.0,0.142857,5.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df = df.interpolate(method='linear', order=2)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 104470 entries, 2011-04-18 13:22:00 to 2011-05-24 19:56:30
Freq: 30S
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Total          104470 non-null  float64
 1   fridge         104470 non-null  float64
 2   dw             104470 non-null  float64
 3   mw             104470 non-null  float64
 4   wm             104470 non-null  float64
 5   fridge_status  104470 non-null  float64
 6   dw_status      104470 non-null  float64
 7   mw_status      104470 non-null  float64
 8   wm_status      104470 non-null  float64
dtypes: float64(9)
memory usage: 8.0 MB


In [13]:
check_nan = df.isnull().sum()
print(check_nan)

Total            0
fridge           0
dw               0
mw               0
wm               0
fridge_status    0
dw_status        0
mw_status        0
wm_status        0
dtype: int64


In [14]:
df.describe()

,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
count,104470.000000,104470.000000,104470.000000,104470.000000,104470.000000,104470.000000,104470.000000,104470.000000,104470.000000
mean,107.491688,63.449813,12.809753,13.171005,18.061117,0.285196,0.009353,0.005939,0.005963
std,276.918926,78.157745,107.188783,109.745024,211.432596,0.388477,0.095285,0.071977,0.074507
min,7.000000,0.285714,0.000000,1.555556,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.573487,6.517931,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,11.000000,7.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,150.166837,130.957788,0.000000,4.000000,0.000000,0.610859,0.000000,0.000000,0.000000
max,4895.833333,523.857143,1161.571429,2103.000000,3277.250000,1.000000,1.000000,1.000000,1.000000


In [15]:
raw3 = df # data after linear interpolation and downsampling to 30 second
df

,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
Time,,,,,,,,,
2011-04-18 13:22:00,11.200000,6.000000,0.200000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:22:30,11.111111,6.000000,0.111111,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:00,11.333333,6.000000,0.333333,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:30,11.250000,6.000000,0.250000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:24:00,11.142857,6.000000,0.142857,5.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2011-05-24 19:54:30,192.095808,188.095808,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
2011-05-24 19:55:00,192.047904,188.047904,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
2011-05-24 19:55:30,192.000000,188.000000,0.000000,4.0,0.0,1.0,0.0,0.0,0.0


In [16]:
df.reset_index(inplace=True)
df

,Time,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
0,2011-04-18 13:22:00,11.200000,6.000000,0.200000,5.0,0.0,0.0,0.0,0.0,0.0
1,2011-04-18 13:22:30,11.111111,6.000000,0.111111,5.0,0.0,0.0,0.0,0.0,0.0
2,2011-04-18 13:23:00,11.333333,6.000000,0.333333,5.0,0.0,0.0,0.0,0.0,0.0
3,2011-04-18 13:23:30,11.250000,6.000000,0.250000,5.0,0.0,0.0,0.0,0.0,0.0
4,2011-04-18 13:24:00,11.142857,6.000000,0.142857,5.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
104465,2011-05-24 19:54:30,192.095808,188.095808,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
104466,2011-05-24 19:55:00,192.047904,188.047904,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
104467,2011-05-24 19:55:30,192.000000,188.000000,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
104468,2011-05-24 19:56:00,192.625000,188.000000,0.625000,4.0,0.0,1.0,0.0,0.0,0.0


In [17]:
# from IPython.display import Image, display
# mpl.rcParams['figure.figsize'] = (25,13)
# mpl.rcParams['axes.grid'] = False
# df.set_index('Time')[['Total']].plot(subplots=True)

In [18]:
# df.set_index('Time')[['fridge']].plot(subplots=True)

In [19]:
# df.set_index('Time')[['dw']].plot(subplots=True)

In [20]:
# df.set_index('Time')[['mw']].plot(subplots=True)

In [21]:
# df.set_index('Time')[['wm']].plot(subplots=True)

In [22]:
df['Year'] = df['Time'].dt.year
df['Month']=df['Time'].dt.month
df['Quarter_of_year']=df['Time'].dt.quarter
df['Day_of_week']=df['Time'].dt.day_of_week
df['Week_of_year']=df['Time'].dt.week
df['Day']=df['Time'].dt.day
df['Hour']=df['Time'].dt.hour
df['Minute']=df['Time'].dt.minute
df['Second']=df['Time'].dt.second

# df.drop('Time', axis=1, inplace=True)
df = df.set_index('Time')
df = df[['Year','Quarter_of_year','Month','Week_of_year','Day_of_week','Day','Hour','Minute','Second','Total','fridge','dw','mw','wm','fridge_status','dw_status','mw_status', 'wm_status']]

df

,Year,Quarter_of_year,Month,Week_of_year,Day_of_week,Day,Hour,Minute,Second,Total,fridge,dw,mw,wm,fridge_status,dw_status,mw_status,wm_status
Time,,,,,,,,,,,,,,,,,,
2011-04-18 13:22:00,2011,2,4,16,0,18,13,22,0,11.200000,6.000000,0.200000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:22:30,2011,2,4,16,0,18,13,22,30,11.111111,6.000000,0.111111,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:00,2011,2,4,16,0,18,13,23,0,11.333333,6.000000,0.333333,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:23:30,2011,2,4,16,0,18,13,23,30,11.250000,6.000000,0.250000,5.0,0.0,0.0,0.0,0.0,0.0
2011-04-18 13:24:00,2011,2,4,16,0,18,13,24,0,11.142857,6.000000,0.142857,5.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-05-24 19:54:30,2011,2,5,21,1,24,19,54,30,192.095808,188.095808,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
2011-05-24 19:55:00,2011,2,5,21,1,24,19,55,0,192.047904,188.047904,0.000000,4.0,0.0,1.0,0.0,0.0,0.0
2011-05-24 19:55:30,2011,2,5,21,1,24,19,55,30,192.000000,188.000000,0.000000,4.0,0.0,1.0,0.0,0.0,0.0


In [23]:
df_resample = df
df_resample.drop(df_resample.columns[[1,2,3,4,10, 11,12,13, 15, 16,17]], axis=1, inplace=True)
df_resample.shape

(104470, 7)

In [24]:
df_resample['fridge_status'] = df_resample['fridge_status'].apply(np.ceil)
df

,Year,Day,Hour,Minute,Second,Total,fridge_status
Time,,,,,,,
2011-04-18 13:22:00,2011,18,13,22,0,11.200000,0.0
2011-04-18 13:22:30,2011,18,13,22,30,11.111111,0.0
2011-04-18 13:23:00,2011,18,13,23,0,11.333333,0.0
2011-04-18 13:23:30,2011,18,13,23,30,11.250000,0.0
2011-04-18 13:24:00,2011,18,13,24,0,11.142857,0.0
...,...,...,...,...,...,...,...
2011-05-24 19:54:30,2011,24,19,54,30,192.095808,1.0
2011-05-24 19:55:00,2011,24,19,55,0,192.047904,1.0
2011-05-24 19:55:30,2011,24,19,55,30,192.000000,1.0


In [25]:
df_resample.fridge_status.unique()

array([0., 1.])

### Creating Labeled, Unlabeled and Test data

In [26]:
train = df_resample.iloc[:79260]
un = df_resample.iloc[79200:91260]
train_un = df_resample.iloc[:91260]
test = df_resample.iloc[91260:101060]

In [27]:
X_train = train[['Year','Day','Hour','Minute','Second','Total']] # Mind, this makes a copy.
y_train = train[['fridge_status']]

X_un = un[['Year','Day','Hour','Minute','Second','Total']] # Mind, this makes a copy.
y_un = un[['fridge_status']]

X_train_un = train_un[['Year','Day','Hour','Minute','Second','Total']] # Mind, this makes a copy.
y_train_un = train_un[['fridge_status']]

X_test = test[['Year','Day','Hour','Minute','Second','Total']] # Mind, this makes a copy.
y_test = test[['fridge_status']]

X_train = X_train.values
X_un = X_un.values
X_train_un = X_train_un.values
X_test = X_test.values

print(y_train.shape, y_un.shape, y_train_un.shape, y_test.shape)

(79260, 1) (12060, 1) (91260, 1) (9800, 1)


In [28]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_test = scaler.fit_transform(X_test)
X_un = scaler.fit_transform(X_un)
X_train_un = scaler.fit_transform(X_train_un)
X_train = scaler.fit_transform(X_train)

In [29]:
timesteps=60
timestep_data = []
for i in range(len(X_train) - timesteps):
    timestep_data.append(X_train[i:i + timesteps])

X_train = np.array(timestep_data)

In [30]:
timesteps=60
timestep_data = []
for i in range(len(X_un) - timesteps):
    timestep_data.append(X_un[i:i + timesteps])

X_un = np.array(timestep_data)

In [31]:
timesteps=60
timestep_data = []
for i in range(len(X_train_un) - timesteps):
    timestep_data.append(X_train_un[i:i + timesteps])

X_train_un = np.array(timestep_data)

In [32]:
timesteps=60
timestep_data = []
for i in range(len(X_test) - timesteps):
    timestep_data.append(X_test[i:i + timesteps])

X_test = np.array(timestep_data)

In [33]:
print('Labeled X shape is: ',X_train.shape,' ; Unlabeled X shape: ',X_un.shape,' ; Combined labled and unlabeled X shape:',X_train_un.shape,' ; X Test shape is: ',X_test.shape)

Labeled X shape is:  (79200, 60, 6)  ; Unlabeled X shape:  (12000, 60, 6)  ; Combined labled and unlabeled X shape: (91200, 60, 6)  ; X Test shape is:  (9740, 60, 6)


In [34]:
y_train = y_train.iloc[:79200]
y_un = y_un.iloc[:12000]
y_train_un = y_train_un.iloc[:91200]
y_test = y_test.iloc[:9740]


y_train = y_train.values
y_un = y_un.values
y_train_un = y_train_un.values
y_test = y_test.values


print('Labeled Y shape is: ',y_train.shape,' ; Unlabeled Y shape: ',y_un.shape,' ; Combined labled and unlabeled Y shape:',y_train_un.shape,' ; Y Test shape is: ',y_test.shape)

Labeled Y shape is:  (79200, 1)  ; Unlabeled Y shape:  (12000, 1)  ; Combined labled and unlabeled Y shape: (91200, 1)  ; Y Test shape is:  (9740, 1)


### Creating the SSL TCN-LSTM model

########   Student Network   #########

In [35]:
n_timesteps, n_features, n_outputs = 60, 6, 1

model = Sequential([
    LSTM(32, input_shape=(n_timesteps, n_features),return_sequences=True),
    TCN(64, 3, input_shape=(n_timesteps, n_features),activation='relu',dilations=(1, 2, 4, 8, 32),return_sequences=True),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
2475/2475 [==============================] - 70s 28ms/step - loss: 0.1490 - accuracy: 0.9350
Epoch 2/20
2475/2475 [==============================] - 71s 29ms/step - loss: 0.0702 - accuracy: 0.9701
Epoch 3/20
2475/2475 [==============================] - 77s 31ms/step - loss: 0.0570 - accuracy: 0.9766
Epoch 4/20
2475/2475 [==============================] - 77s 31ms/step - loss: 0.0462 - accuracy: 0.9814
Epoch 5/20
2475/2475 [==============================] - 77s 31ms/step - loss: 0.0395 - accuracy: 0.9843
Epoch 6/20
2475/2475 [==============================] - 74s 30ms/step - loss: 0.0361 - accuracy: 0.9860
Epoch 7/20
2475/2475 [==============================] - 76s 31ms/step - loss: 0.0310 - accuracy: 0.9879
Epoch 8/20
2475/2475 [==============================] - 83s 33ms/step - loss: 0.0278 - accuracy: 0.9891
Epoch 9/20
2475/2475 [==============================] - 80s 32ms/step - loss: 0.0248 - accuracy: 0.9902
Epoch 10/20
2475/2475 [==============================] - 79s 32m

########   Teacher Network   #########

In [36]:
n_timesteps, n_features, n_outputs = 60, 6, 1

model2 = Sequential([
    LSTM(32, input_shape=(n_timesteps, n_features),return_sequences=True),
    TCN(64, 3, input_shape=(n_timesteps, n_features),activation='relu',dilations=(1, 2, 4, 8, 32),return_sequences=True),
    Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.fit(X_train_un, y_train_un, epochs=20, batch_size=32)

Epoch 1/20
2850/2850 [==============================] - 89s 31ms/step - loss: 0.1641 - accuracy: 0.9246
Epoch 2/20
2850/2850 [==============================] - 89s 31ms/step - loss: 0.0600 - accuracy: 0.9740
Epoch 3/20
2850/2850 [==============================] - 90s 32ms/step - loss: 0.0497 - accuracy: 0.9790
Epoch 4/20
2850/2850 [==============================] - 93s 33ms/step - loss: 0.0414 - accuracy: 0.9832
Epoch 5/20
2850/2850 [==============================] - 89s 31ms/step - loss: 0.0332 - accuracy: 0.9866
Epoch 6/20
2850/2850 [==============================] - 90s 32ms/step - loss: 0.0295 - accuracy: 0.9882
Epoch 7/20
2850/2850 [==============================] - 89s 31ms/step - loss: 0.0259 - accuracy: 0.9897
Epoch 8/20
2850/2850 [==============================] - 90s 32ms/step - loss: 0.0238 - accuracy: 0.9905
Epoch 9/20
2850/2850 [==============================] - 89s 31ms/step - loss: 0.0217 - accuracy: 0.9913
Epoch 10/20
2850/2850 [==============================] - 92s 32m

In [37]:
y_pred=model2.predict(X_un)
y_predict = [np.round(x[0]) for x in y_pred]

In [38]:
np.concatenate([y_train, y_predict])

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

########   Updated and Final Student Network (It will be the final SSL model)  #########

In [39]:
history = model.fit(X_train_un, np.concatenate([y_train, y_predict]), epochs=20, batch_size=32)

Epoch 1/20
2850/2850 [==============================] - 82s 29ms/step - loss: 0.0419 - accuracy: 0.9830
Epoch 2/20
2850/2850 [==============================] - 73s 26ms/step - loss: 0.0279 - accuracy: 0.9891
Epoch 3/20
2850/2850 [==============================] - 70s 25ms/step - loss: 0.0265 - accuracy: 0.9897
Epoch 4/20
2850/2850 [==============================] - 70s 25ms/step - loss: 0.0236 - accuracy: 0.9907
Epoch 5/20
2850/2850 [==============================] - 70s 24ms/step - loss: 0.0218 - accuracy: 0.9917
Epoch 6/20
2850/2850 [==============================] - 70s 24ms/step - loss: 0.0213 - accuracy: 0.9918
Epoch 7/20
2850/2850 [==============================] - 70s 24ms/step - loss: 0.0226 - accuracy: 0.9914
Epoch 8/20
2850/2850 [==============================] - 69s 24ms/step - loss: 0.0180 - accuracy: 0.9927
Epoch 9/20
2850/2850 [==============================] - 72s 25ms/step - loss: 0.0195 - accuracy: 0.9925
Epoch 10/20
2850/2850 [==============================] - 74s 26m

In [41]:
loss, accuracy = history.model.evaluate(X_test, y_test)

305/305 [==============================] - 3s 10ms/step - loss: 2.2046 - accuracy: 0.5067


In [42]:
print("Loss is: ",loss, " and Accuracy is: ",accuracy )

Loss is:  2.204576253890991  and Accuracy is:  0.5067147016525269


In [43]:
y_pred=history.model.predict(X_test)
y_predict = [np.round(x[0]) for x in y_pred]

In [44]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.5489790643577153